In [1]:

import matplotlib
matplotlib.use('Agg')
import os
import datetime
import numpy as np
import dill as pickle
import random
import sys
import seaborn as sns
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from collections import OrderedDict
import math
from matplotlib.lines import Line2D
from pylab import rcParams
from collections import Counter
from itertools import combinations

from shutil import copyfile
rcParams['figure.figsize'] = 25, 10

np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)
from codes.experimentclasses.RigidEthanolPCA import RigidEthanolPCA
from codes.otherfunctions.get_dictionaries import get_all_atoms_4
from codes.otherfunctions.get_grads import get_grads
from codes.otherfunctions.multirun import get_support_recovery_lambda
from codes.otherfunctions.multirun import get_lower_interesting_lambda
from codes.otherfunctions.multirun import get_coeffs_and_lambdas
from codes.otherfunctions.multirun import get_support
from codes.otherfunctions.multiplot import plot_support_2d
from codes.otherfunctions.multiplot import plot_reg_path_ax_lambdasearch
from codes.otherfunctions.multiplot import plot_gs_v_dgnorm
from codes.otherfunctions.multiplot import plot_dot_distributions
from codes.otherfunctions.multirun import get_cosines
from codes.flasso.Replicate import Replicate
from codes.otherfunctions.multirun import get_olsnorm_and_supportsbrute
from codes.otherfunctions.multiplot import highlight_cell

#set parameters
n = 10000 #number of data points to simulate
nsel = 100 #number of points to analyze with lasso
itermax = 1000 #maximum iterations per lasso run
tol = 1e-10 #convergence criteria for lasso
#lambdas = np.asarray([0,.01,.1,1,10,100], dtype = np.float16)#lambda values for lasso
lambdas = np.asarray(np.hstack([np.asarray([0]),np.logspace(-3,1,11)]), dtype = np.float16)
n_neighbors = 1000 #number of neighbors in megaman
m = 3 #number of embedding dimensions (diffusion maps)
#diffusion_time = 1. #diffusion time controls gaussian kernel radius per gradients paper
diffusion_time = 0.05 #(yuchia suggestion)
dim = 2 #manifold dimension
dimnoise = 2
natoms = 9
cores = 3 #number of cores for parallel processing
cor = 0.0 #correlation for noise
var = 0.00001 #variance scaler for noise
cor = 0.0 #correlation for noise
var = 0.00001 #variance scaler for noise
ii = np.asarray([0,0,0,0,1,1,1,2]) # atom adjacencies for dihedral angle computation
jj = np.asarray([1,2,3,4,5,6,7,8])

#run experiment
atoms4 = np.asarray([[6,1,0,4],[4,0,2,8],[7,6,5,1],[3,0,2,4]],dtype = int)
nreps = 25
lambda_max = 1
max_search = 30

new_MN = True
new_grad = True
savename = 'rigidethanol_110120_alltorsions'
savefolder = 'rigidethanol'
loadfolder = 'rigidethanol'
loadname = 'rigidethanol_110120_alltorsions'

/Users/samsonkoelle/manifoldflasso_jmlr


In [2]:
with open(workingdirectory + '/untracked_data/embeddings/re_test_exp.pkl' ,
         'rb') as file:
     experiment = pickle.load(file)

with open(workingdirectory + '/untracked_data/embeddings/re_test_replicate.pkl' ,
         'rb') as file:
     replicate= pickle.load(file)

In [3]:
X = replicate.xtrain
Y = replicate.ytrain
group = replicate.groups

In [54]:
#The true champion

import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad
from autograd import grad

import logging
from copy import deepcopy

#import numpy as np
from scipy.special import expit
from pyglmnet import utils


class GLM:
    
    def __init__(self, xs, ys, reg_lambda, group,max_iter, learning_rate, tol,parameter):
        self.xs = xs
        self.ys = ys
        self.reg_lambda = reg_lambda
        self.group = np.asarray(group)
        #print(self.group.shape)
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.tol = tol
        self.Tau = None
        self.alpha = 1.
        self.lossresults = {}
        self.dls = {}
        self.parameter = parameter
        self.l2loss = {}
        self.penalty = {}
        
    def _prox(self,beta, thresh):
        """Proximal operator."""
        
        #print(thresh, beta)
        #print('beginprox', beta[0:2],thresh)
        group_ids = np.unique(self.group)
        result = np.zeros(beta.shape)
        result = np.asarray(result, dtype = float)
        #print('gids',group_ids)
        for i in range(len(group_ids)):
            gid = i 
            #print(self.group)
            idxs_to_update = np.where(self.group == gid)[0]
            #print('idx',idxs_to_update)
            #print('norm', np.linalg.norm(beta[idxs_to_update]))
            if np.linalg.norm(beta[idxs_to_update]) > 0.:
                #print('in here', len(idxs_to_update))
                potentialoutput = beta[idxs_to_update] - (thresh / np.linalg.norm(beta[idxs_to_update])) * beta[idxs_to_update]
                posind = np.where(beta[idxs_to_update] > 0.)[0]
                negind = np.where(beta[idxs_to_update] < 0.)[0]
                po = beta[idxs_to_update].copy()
                #print('potention', potentialoutput[0:2])
                po[posind] = np.asarray(np.clip(potentialoutput[posind],a_min = 0., a_max = 1e15), dtype = float)
                po[negind] = np.asarray(np.clip(potentialoutput[negind],a_min = -1e15, a_max = 0.), dtype = float)
                result[idxs_to_update] = po
        #print('end', result[0:2])
        return result

    def _grad_L2loss(self, beta, X, y):
        #print(beta.shape,X.shape,y.shape)
        if y.ndim == 1:
            y = y[:, np.newaxis]
        n_samples = np.float(X.shape[0])
        z = np.dot(X, beta)
        #grad_beta = 1. / n_samples * np.transpose(np.dot(np.transpose(z - y), X))
        grad_beta = np.transpose(np.dot(np.transpose(z - y), X))
        #print('gb',grad_beta.shape)
        return grad_beta
    
    def _loss(self,beta, reg_lambda, X, y):
        """Define the objective function for elastic net."""
        L = self._logL(beta, X, y)
        P = self._L1penalty(beta)
        J = -L + reg_lambda * P
        return J
    
    def _logL(self,beta, X, y):
        """The log likelihood."""
        #print('beginlogL', np.linalg.norm(beta), np.linalg.norm(X), np.linalg.norm(y),y.shape,beta.shape,X.shape,)
        l = np.dot(X, beta)
        logL = -0.5 * np.sum((y - l)**2)
        #print('endlogL',logL)
        return logL
    
    def _L2loss(self,beta,X,y):
        #print('beginl2', np.linalg.norm(beta), np.linalg.norm(X), np.linalg.norm(y), y.shape)
        output = -self._logL(beta, X, y)
        #print('outl2',output)
        return(output)
    
    def _L1penalty(self, beta):
        """The L1 penalty"""
        # Compute the L1 penalty
        group = self.group
        group_ids = np.unique(self.group)
        L1penalty = 0.0
        for group_id in group_ids:
            L1penalty += np.linalg.norm(beta[self.group == group_id], 2)
        return L1penalty
    
    #def fhatlambda(self,lamb,x,y):
    def fhatlambda(self,lamb,betanew,betaold):
        xs = self.xs
        ys = self.ys
        #print(ys.shape,'fhatlam')
        #print(self._L2loss(betaold,xs,ys),self._L2loss(betanew,xs,ys),'old','new') 
        output = self._L2loss(betaold,xs,ys) + np.dot(self._grad_L2loss(betaold,xs,ys).transpose(),(betanew-betaold)) + (1/(2*lamb)) * np.linalg.norm(betanew-betaold)**2
        return(output)
    
    #_btalgorithm(yk,lamb,.5,1000, rl)
    def _btalgorithm(self,bet,lam,b,maxx,rl):
        
        #print('lam',lam)
        X = self.xs
        y = self.ys
        #print('beginbt', np.linalg.norm(y))
        #print('beginbt',self._L2loss(bet,X,y))
        #print(np.linalg.norm(bet))
        grad_beta = self._grad_L2loss(beta = bet, X = X, y = y)
        for i in range(maxx):
            #print('bt',i)
            betn = bet - lam * grad_beta
            z = self._prox(betn, lam * rl)
            fz = self._L2loss(z,X,y)
            #print(fz,'fz')
            fhatz = self.fhatlambda(lam,z, bet)
            if fz <= fhatz:
                print(i,'btiterstop')
            #print(fhatz - fz)
            #if 0 <= 1:
                break
            lam = b*lam
        return(z,lam)
    
    def fit(self):
        

        group  = self.group
        print(group.shape)
        lambdas = self.reg_lambda
        parameter = self.parameter
        X = self.xs
        #print(X.shape)
        y = self.ys
        
        np.random.RandomState(0)
        group = np.asarray(group, dtype = np.int64)

        #print(group.shape[0])
        group.dtype = np.int64
        #print(group.shape[0])
        #print(X.shape[1])
        if group.shape[0] != X.shape[1]:
            raise ValueError('group should be (n_features,)')

        # type check for data matrix
        if not isinstance(X, np.ndarray):
            raise ValueError('Input data should be of type ndarray (got %s).'
                             % type(X))

        n_features = np.float(X.shape[1])
        n_features = np.int64(n_features)
        if y.ndim == 1:
            y = y[:, np.newaxis]
            self.ys = y
        #print(y.shape)
        n_classes = 1
        n_classes = np.int64(n_classes)

        beta_hat = 1 / (n_features) * np.random.normal(0.0, 1.0, [n_features, n_classes])
        fit_params = list()
        
        for l, rl in enumerate(lambdas):
            fit_params.append({'beta': beta_hat})
            if l == 0:
                fit_params[-1]['beta'] = beta_hat
            else:
                fit_params[-1]['beta'] = fit_params[-2]['beta']
            tol = self.tol
            alpha = 1.
            beta = np.zeros([n_features, n_classes])
            beta = fit_params[-1]['beta']
            #print('losser',self._L2loss(beta,X,y))
            g = np.zeros([n_features, n_classes])
            L, DL ,L2,PEN = list(), list() , list(), list()
            lamb = self.learning_rate
            bm1 = beta.copy()
            bm2 = beta.copy()
            for t in range(0, self.max_iter):
                L.append(self._loss(beta, rl, X, y))
                L2.append(self._L2loss(beta,X,y))
                PEN.append(self._L1penalty(beta))
                w = (t / (t+ 3))
                yk = beta + w*(bm1 - bm2)
                #print('losser',self._L2loss(yk,X,y))
                #print('beforebt',np.linalg.norm(yk),np.linalg.norm(X),np.linalg.norm(y))
                beta , lamb = self._btalgorithm(yk,lamb,.5,1000, rl)
                #X = self.xs
                #y = self.ys
                #print('losser2',self._L2loss(beta,X,y))
                bm2 = bm1.copy()
                bm1 = beta.copy()
                if t > 1:
                    DL.append(L[-1] - L[-2])
                    if np.abs(DL[-1] / L[-1]) < tol:
                        print('converged', rl)
                        msg = ('\tConverged. Loss function:'
                               ' {0:.2f}').format(L[-1])
                        msg = ('\tdL/L: {0:.6f}\n'.format(DL[-1] / L[-1]))
                        break
                    
            #print(beta)
            fit_params[-1]['beta'] = beta
            self.lossresults[rl] = L
            self.l2loss[rl] = L2
            self.penalty[rl] = PEN
            self.dls[rl] = DL
            #print(L)
        # Update the estimated variables
        
        self.fit_ = fit_params
        self.ynull_ = np.mean(y)

        # Return
        return self
 
#The true champion

# import sys
# !{sys.executable} -m pip install einops
from einops import rearrange

import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad
from autograd import grad

import logging
from copy import deepcopy

#import numpy as np
from scipy.special import expit
from pyglmnet import utils


class GradientGroupLasso:
    
    def __init__(self, dg_M, df_M, reg_l1s, reg_l2, max_iter,learning_rate, tol, beta0_npm= None):
        
        n = dg_M.shape[0]
        d= dg_M.shape[1]
        m = df_M.shape[2]
        p = dg_M.shape[2]
        dummy_beta = np.ones((n,p,m))
        
        self.dg_M = dg_M
        self.df_M = df_M
        self.reg_l1s = reg_l1s
        self.reg_l2 = reg_l2
        self.beta0_npm = beta0_npm
        self.n = n
        self.p = p
        self.m = m 
        self.d = d
        self.dummy_beta = dummy_beta
        self.group = np.asarray(group)
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.tol = tol
        self.Tau = None
        self.alpha = 1.
        self.lossresults = {}
        self.dls = {}
        self.l2loss = {}
        self.penalty = {}
        
    def _prox(self,beta_npm, thresh):
        """Proximal operator."""
        
        p = self.p
        result = np.zeros(beta_npm.shape)
        result = np.asarray(result, dtype = float)
        for j in range(p):
            if np.linalg.norm(beta_npm[:,j,:]) > 0.:
                potentialoutput = beta_npm[:,j,:] - (thresh / np.linalg.norm(beta_npm[:,j,:])) * beta_npm[:,j,:]
                posind = np.asarray(np.where(beta_npm[:,j,:] > 0.))
                negind = np.asarray(np.where(beta_npm[:,j,:] < 0.))
                po = beta_npm[:,j,:].copy()
                po[posind[0],posind[1]] = np.asarray(np.clip(potentialoutput[posind[0],posind[1]],a_min = 0., a_max = 1e15), dtype = float)
                po[negind[0],negind[1]] = np.asarray(np.clip(potentialoutput[negind[0],negind[1]],a_min = -1e15, a_max = 0.), dtype = float)
                result[:,j,:] = po
        return result

    def _grad_L2loss(self, beta_npm):
        
        df_M = self.df_M
        dg_M = self.dg_M
        reg_l2 = self.reg_l2
        dummy_beta = self.dummy_beta
        
        df_M_hat = np.einsum('ndp,npm->ndm',dg_M, beta_npm)
        error = df_M_hat - df_M
        grad_beta = np.einsum('ndm,ndp->npm',error,dg_M) #+ reg_l2 * np.ones()
        #if 
        return grad_beta
    
    def _L1penalty(self, beta_npm):
        
        p = self.p
        m = self.m
        n = self.n 
        beta_mn_p = rearrange(beta_npm, 'n p m -> (m n) p')#np.reshape(beta_mnp, ((m*n,p)))
        L1penalty = np.linalg.norm(beta_mn_p, axis = 0).sum()
        
        return L1penalty
    
    def _loss(self,beta_npm, reg_lambda):
        """Define the objective function for elastic net."""
        L = self._logL(beta_npm)
        P = self._L1penalty(beta_npm)
        J = -L + reg_lambda * P
        return J
    
    def _logL(self,beta_npm):
        
        df_M = self.df_M
        dg_M = self.dg_M
        
        df_M_hat = np.einsum('ndp,npm -> ndm',dg_M, beta_npm)
        logL = -0.5 * np.linalg.norm((df_M - df_M_hat))**2
        return(logL)
    
    def _L2loss(self,beta_npm):
        output = -self._logL(beta_npm)
        return(output)
    
    
    #def fhatlambda(self,lamb,x,y):
    def fhatlambda(self,learning_rate,beta_npm_new,beta_npm_old):
        #output = self._L2loss(beta_npm_old) + np.linalg.norm(self._grad_L2loss(beta_npm_old) * (beta_npm_new-beta_npm_old))**2 + (1/(2*learning_rate)) * np.linalg.norm(beta_npm_new-beta_npm_old)**2
        output = self._L2loss(beta_npm_old) + np.einsum('npm,npm', self._grad_L2loss(beta_npm_old),(beta_npm_new-beta_npm_old)) + (1/(2*learning_rate)) * np.linalg.norm(beta_npm_new-beta_npm_old)**2
        return(output)

#         def fhatlambda(self,lamb,betanew,betaold):
#         xs = self.xs
#         ys = self.ys
#         #print(ys.shape,'fhatlam')
#         #print(self._L2loss(betaold,xs,ys),self._L2loss(betanew,xs,ys),'old','new') 
#         output = self._L2loss(betaold,xs,ys) + np.dot(self._grad_L2loss(betaold,xs,ys).transpose(),(betanew-betaold)) + (1/(2*lamb)) * np.linalg.norm(betanew-betaold)**2
#         return(output)
    #_btalgorithm(yk,lamb,.5,1000, rl)
    def _btalgorithm(self,beta_npm ,learning_rate,b,maxiter_bt,rl):
        
        grad_beta = self._grad_L2loss(beta_npm = beta_npm)
        for i in range(maxiter_bt):
            beta_npm_postgrad = beta_npm - learning_rate * grad_beta
            beta_npm_postgrad_postprox = self._prox(beta_npm_postgrad, learning_rate * rl)
            fz = self._L2loss(beta_npm_postgrad_postprox)
            #fhatz = self.fhatlambda(lam,beta_npm_postgrad_postprox, beta_npm_postgrad)
            fhatz = self.fhatlambda(learning_rate,beta_npm_postgrad_postprox, beta_npm)
            if fz <= fhatz:
                #print(i)
                break
            learning_rate = b*learning_rate    
            
        return(beta_npm_postgrad_postprox,learning_rate)
    
    def fit(self, beta0_npm = None):

        reg_l1s = self.reg_l1s
        n = self.n
        m = self.m
        p = self.p
        
        dg_M = self.dg_M
        df_M = self.df_M
        
        tol = self.tol
        np.random.RandomState(0)
        
        if beta0_npm != None:
            beta_npm_hat = beta0_npm #1 / (n_features) * np.random.normal(0.0, 1.0, [n_features, n_classes])
        else: 
            beta_npm_hat = 1 / (n*m*p) * np.random.normal(0.0, 1.0, [n, p,m])
            
        fit_params = list()
        for l, rl in enumerate(reg_l1s):
            fit_params.append({'beta': beta_npm_hat})
            if l == 0:
                fit_params[-1]['beta'] = beta_npm_hat
            else:
                fit_params[-1]['beta'] = fit_params[-2]['beta']
            
            alpha = 1.
            beta_npm_hat = fit_params[-1]['beta']
            #g = np.zeros([n_features, n_classes])
            L, DL ,L2,PEN = list(), list() , list(), list()
            learning_rate = self.learning_rate
            beta_npm_hat_1 = beta_npm_hat.copy()
            beta_npm_hat_2 = beta_npm_hat.copy()
            for t in range(0, self.max_iter):
                #print(t,l,rl)
                L.append(self._loss(beta_npm_hat, rl))
                L2.append(self._L2loss(beta_npm_hat))
                PEN.append(self._L1penalty(beta_npm_hat))
                w = (t / (t+ 3))
                beta_npm_hat_momentumguess = beta_npm_hat + w*(beta_npm_hat_1 - beta_npm_hat_2)
                
                beta_npm_hat , learning_rate = self._btalgorithm(beta_npm_hat_momentumguess,learning_rate,.5,1000, rl)
                #print(beta_npm_hat_momentumguess.max(), beta_npm_hat.max(),self._L2loss(beta_npm_hat), learning_rate)
                beta_npm_hat_2 = beta_npm_hat_1.copy()
                beta_npm_hat_1 = beta_npm_hat.copy()
                
                if t > 1:
                    DL.append(L[-1] - L[-2])
                    if np.abs(DL[-1] / L[-1]) < tol:
                        print('converged', rl)
                        msg = ('\tConverged. Loss function:'
                               ' {0:.2f}').format(L[-1])
                        msg = ('\tdL/L: {0:.6f}\n'.format(DL[-1] / L[-1]))
                        break

            fit_params[-1]['beta'] = beta_npm_hat
            self.lossresults[rl] = L
            self.l2loss[rl] = L2
            self.penalty[rl] = PEN
            self.dls[rl] = DL

        self.fit_ = fit_params
        #self.ynull_ = np.mean(y)

        return self
    

In [64]:
def get_sr_lambda_sam(dg_M, df_M, gl_itermax, lambdas_start,reg_l2, max_search, card, tol,learning_rate):
    
    highprobes = np.asarray([])
    lowprobes = np.asarray([])
    #lambdas_start = np.asarray([0,1])
    probe_init_low = lambdas_start[0]
    probe_init_high = lambdas_start[1]

    
    coeffs = {}
    combined_norms = {}

    GGL = GradientGroupLasso(dg_M, df_M, np.asarray([probe_init_low]), reg_l2, gl_itermax,learning_rate, tol, beta0_npm= None)
    GGL.fit()
    beta0_npm = GGL.fit_[-1]['beta']
    coeffs[probe_init_low] = GGL.fit_[-1]['beta']
    combined_norms[probe_init_low] = np.sqrt((np.linalg.norm(coeffs[probe_init_low], axis = 0)**2).sum(axis = 1))

    GGL = GradientGroupLasso(dg_M, df_M, np.asarray([probe_init_high]), reg_l2, gl_itermax,learning_rate, tol, beta0_npm= beta0_npm)
    GGL.fit()
    #beta0_npm = GGL.fit_[-1]['beta']
    coeffs[probe_init_high] = GGL.fit_[-1]['beta']
    combined_norms[probe_init_high] = np.sqrt((np.linalg.norm(coeffs[probe_init_high], axis = 0)**2).sum(axis = 1))

    n_comp = len(np.where(combined_norms[probe_init_high] != 0)[0])
    lowprobes = np.append(lowprobes, probe_init_low)

    if n_comp == card:
        #high_int = probe
        print('we did it')
        return (probe_init_high, coeffs, combined_norms)

    if n_comp < card:
        highprobes = np.append(highprobes, probe_init_high)
        probe = (lowprobes.max() + highprobes.min()) / 2
    if n_comp > card:
        lowprobes = np.append(lowprobes, probe_init_high)
        probe = lowprobes.max() * 2

    for i in range(max_search):
        print(i, probe, 'probe')
        beta0_npm = coeffs[lowprobes.max()]
        if not np.isin(probe, list(combined_norms.keys())):
            #print('probe',probe)
            GGL = GradientGroupLasso(dg_M, df_M, np.asarray([probe]), reg_l2, gl_itermax,learning_rate, tol, beta0_npm = beta0_npm)
            GGL.fit()
            coeffs[probe] = GGL.fit_[-1]['beta']
            combined_norms[probe] = np.sqrt((np.linalg.norm(coeffs[probe], axis = 0)**2).sum(axis = 1))

        n_comp = len(np.where(combined_norms[probe] != 0)[0])
        if n_comp == card:
            #high_int = probe
            print('we did it')
            return (probe, coeffs, combined_norms)
        else:
            if n_comp < card:
                highprobes = np.append(highprobes, probe)
            if n_comp > card:
                lowprobes = np.append(lowprobes, probe)
            if len(highprobes) > 0:
                probe = (lowprobes.max() + highprobes.min()) / 2
            else:
                probe = lowprobes.max() * 2

                
    #return (probe, coeffs, combined_norms)

In [67]:
dg_M = np.swapaxes(replicate.dg_M,1,2)
df_M = replicate.df_M
dg_M = dg_M[:1]
df_M = df_M[:1]
gl_itermax = 500
lambdas_start = [0.,.001]
max_search = 15
reg_l2 = 0.
card = 2
tol = 1e-8
learning_rate = .1

In [68]:
get_sr_lambda_sam(dg_M, df_M, gl_itermax, lambdas_start, reg_l2,max_search, card, tol,learning_rate)

0 0.002 probe
converged 0.002
1 0.004 probe
2 0.008 probe
converged 0.008
3 0.006 probe
converged 0.006
4 0.005 probe
5 0.0045000000000000005 probe
we did it


(0.0045000000000000005,
 {0.0: array([[[-2.01485513e-04,  2.15496333e-04, -1.62891494e-04],
          [ 2.72306261e-04,  6.52185421e-05, -2.27025033e-04],
          [-1.24142717e-04, -1.56428911e-04,  9.53309570e-04],
          ...,
          [ 3.28645570e-04, -3.00232986e-04,  2.91564622e-04],
          [-2.62594601e-04,  6.08943415e-04,  4.18898802e-04],
          [ 4.83216040e-04, -2.76867631e-04,  1.95514054e-04]]]),
  0.001: array([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
  0.002: array([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
  0.004: array([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
  0.0045000000000000005: array([[[0., 0., 0.],
          [0., 0., 0.

In [5]:




def get_supportrecovery_lambda_spams(experiment, replicate, lambda_max, max_search, dim):
    coeffs = {}
    #combined_norms = replicate.combined_norms
    #nsel = replicate.nsel
    probe = 0.

    tol = experiment.tol
    itermax = experiment.itermax
    lambda_max = lambda_max

    for i in range(max_search):
        print(i, probe)
        if not np.isin(probe, list(combined_norms.keys())):
            print('probe',probe)
            coeffs[probe] = experiment.get_betas_spam2(replicate.xtrain, replicate.ytrain, replicate.groups,
                                                            np.asarray([probe]), nsel, experiment.m, itermax, tol)

            combined_norms[probe] = np.linalg.norm(np.linalg.norm(coeffs[probe][:, :, :, :], axis=2), axis=1)[
                                         0, :]
        n_comp = len(np.where(combined_norms[probe] != 0)[0])
        if n_comp == dim:
            high_int = probe
            print('we did it')
            return (high_int, coeffs, combined_norms)
        else:
            if n_comp < dim:
                max_lam = np.min(max_lam, probe)
                #high_probe = high_probe / 2
            if n_comp > dim:
                min_lam = np.max(min_lam, probe)
                if len(max_lam) == 0:
                    probe = probe * 2
                else:
                    probe = (min_lam + max_lam)/2
                #high_probe = high_probe * 1.5
    # return(lambda_max, coeffs,combined_norms)
    return (probe, coeffs, combined_norms)

NameError: name 'glm' is not defined